In [1]:
# créer un nouvel env conda à partir du terminal
# conda create --name pathrag python=3.10
# installer ollama: https://www.ollama.com/download

# installer les dépendences
#%pip install -r requirements.txt
# intsaller le modème d'OllamaEmbeddings
!ollama pull embeddinggemma
# créer une clé api sur openrouter pour utiliser des llm gratuitement

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 0800cbac9c20: 100% ▕██████████████████▏ 621 MB                         
pulling 1adbfec9dcf0: 100% ▕██████████████████▏ 8.4 KB                         
pulling 45dc10444b87: 100% ▕██████████████████▏   34 B                         
pulling 3901c6a1d7c2: 100% ▕██████████████████▏  416 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
from openai import OpenAI, AsyncOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.schema.document import Document
from pathrag_retriever import create_graphdb, load_existing_graphdb, load_knowledgeGraph_vis
import time


/home/chougar/miniconda3/envs/pathrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Base vectorielle
Création de la base vect. avec le PP Mahakam (20 premières pages, ligne 45 `docs[:20]`)

Relancer la cellule à chaque ouverture pour utiliser le rag vectoriel

In [3]:
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.retrievers import TFIDFRetriever
from langchain_unstructured import UnstructuredLoader

#========= choix du modèle d'embedding
"""
    Le modèle choisi impacte la qualité du retriever, mais aussi le temps de traitement
    Si le déploiement est prévu sur une VM limitée, un modèle plus petit est nécessaire
    Explorer les comparatifs: https://huggingface.co/spaces/mteb/leaderboard

"""
# Utiliser OllamaEmbeddings avec le modèle local "embeddinggemma"
embeddings = OllamaEmbeddings(model="embeddinggemma")



# chargement et fragmentation du doc
## Nom du doc à traiter
filename="audio-text.txt"

## Nom pour la base vectorielle
doc_name_hybrid="L-IA-notre-deuxieme-conscience_sample" # nom de doc significatif


# loader = UnstructuredFileLoader(filename)
loader = UnstructuredLoader(filename)

txt_doc = loader.load()
print(f"Loaded {len(txt_doc)} documents from {filename}")


#======== choix des paramètres de fragmentation
"""
    la taille du chunck_size est très important dans l'accès à une info précise
    une plus petite taille permet de cibler de courts passages contenant l'info nécessaire à des réponses précises:
        * lieu du projet
        * dates du projet
        * budget ...    
    l'envoi de passages plus courts au llm évite une dispertion de son attention
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docs = text_splitter.split_documents(txt_doc)

# Filter out complex metadata (e.g., lists, dicts)
docs = [Document(doc.page_content) for doc in docs]


# Conversion des docs en embeddings 
chroma_db = Chroma.from_documents(
    docs,
    embedding=embeddings,
    persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
    collection_name=doc_name_hybrid.replace(" ","_")
)

retriever=chroma_db.as_retriever()





# ...existing code...
all_docs = chroma_db.get()
print("Nb de chuncks:", len(all_docs['documents']))  # This will print the total number of docs stored
# ...existing code...

Loaded 1 documents from audio-text.txt


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


Nb de chuncks: 30


In [4]:
for c in all_docs["documents"]:
    print(c, "\n=============")

[Nathan Devers]: France Culture. Sans préjugés. Natacha Devers. Comment expliquer les progrès phénoménaux que semble avoir accomplis l'intelligence artificielle au cours de ces dernières années ? Depuis l'apparition de ChatGPT en novembre 2022, rapidement suivi par d'autres agents conversationnels. Cette révolution technologique aux multiples aspects, paraît désormais capable d'exécuter de nombreuses tâches intellectuelles sur lesquelles l'esprit humain pensait jusqu'alors exercer un monopole. Écrire des articles, synthétiser des documents, traiter des données dans n'importe quel domaine, diagnostiquer une maladie, rédiger une dissertation, ou pourquoi pas un scénario de film. Non contente de révolutionner le monde du travail, ses prouesses stupéfiantes de la technique soulèvent une interrogation majeure dans le domaine de la philosophie de l'esprit. Faut-il en déduire que l'intelligence, n'étant pas le propre d'un cerveau biologique et encore moins d'une âme, peut s'implémenter dans 


#### Création ou chargement d'un graphe existant:
Saisir l'action désirée dans le prompt qui s'affiche en haut du notebook, suivre instructions

Par défaut prise en compte des 20 premières pages pour l'exemple, modifier ligne 11 `for doc in docx_docs[:20]:`

Si vous voulez charger un graphe déjà crée, retrouver son nom dans le fichier `graphrag_hashes.json`, attribut `Nom du doc`

Si vous voulez modifier le LLM utilisé pour la création du graphe ou sa lecture, allez dans `pathrag_retriever.py`, ligne 34 et 35, et prenez un modèle valide sur openrouter (attention à prendre un modèle qui supporte les `structured_outputs`, à filtrer à droite dans la liste des `Supported parameters`)

**Important**: renseigner votre clé api openrouter sur le script `pathrag_retriever.py`, ligne 42

In [5]:
# appliquer nest_asyncio uniquement sur notebook pour corriger l'erreur de loop event
import nest_asyncio
nest_asyncio.apply()

# remetre à plat le text
filename="audio-text.txt"
loader = UnstructuredLoader(filename)

txt_docs = loader.load()
text=""
for doc in txt_docs:
    text+=doc.page_content


r=input("Saisir 'C' pour créer un nouveau graphe, 'L' pour charger un graphe existant")

# créer un nouveau graphe
messages=None
if r=='C':
    doc_name_graph=input('Saisir un nom unique pour votre graphe')
    print(f"Le nom de votre graphe est {doc_name_graph}")
    messages= create_graphdb(
        text=text, 
        doc_name=doc_name_graph, # il faut donner un nom unique permettant d'identifier et charger le graph les prochaines fois
    )
# charger un graphe existant
elif r=='L':
    doc_name_graph=input('Saisir le nom du graphe à charger')
    print(f"Le nom de votre graphe est {doc_name_graph}")

    messages=load_existing_graphdb(doc_name_graph)
else:
    print('Option invalide')



if messages:
    pipeline_args={}
    for feedback in messages:
        if isinstance(feedback, str):
            print(feedback)
        elif isinstance(feedback, dict):
            pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]=feedback["pipeline_args"]
            


Le nom de votre graphe est L-IA-notre deuxieme-conscience_sample
Génération du hash
load hashes
Chargement de l'historique de hashage Graph RAG
check hash
Nouveau document identifié


INFO:PathRAG:Logger initialized for working directory: /home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/77dbce69c6becbc4ccbea21cb8f1bca1c6e7dc6d24c53f6905ab1af9a73e71ec


Nouveau document identifié
77dbce69c6becbc4ccbea21cb8f1bca1c6e7dc6d24c53f6905ab1af9a73e71ec


INFO:PathRAG:Load KV llm_response_cache with 0 data
INFO:PathRAG:Load KV full_docs with 0 data
INFO:PathRAG:Load KV text_chunks with 0 data
INFO:PathRAG:Loaded graph from /home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/77dbce69c6becbc4ccbea21cb8f1bca1c6e7dc6d24c53f6905ab1af9a73e71ec/graph_chunk_entity_relation.graphml with 0 nodes, 0 edges
INFO:nano-vectordb:Load (0, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/77dbce69c6becbc4ccbea21cb8f1bca1c6e7dc6d24c53f6905ab1af9a73e71ec/vdb_entities.json'} 0 data
INFO:nano-vectordb:Load (0, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/77dbce69c6becbc4ccbea21cb8f1bca1c6e7dc6d24c53f6905ab1af9a73e71ec/vdb_relationships.json'

Création de la chaîne Graph RAG en cours
Temps estimé: 67 secondes
Consommation de tokens estimée: 14275 tokens (90% input / 10% output)


INFO:PathRAG:[New Docs] inserting 1 docs
Chunking documents: 100%|██████████| 1/1 [00:01<00:00,  1.13s/doc]
INFO:PathRAG:[New Chunks] inserting 1 chunks
INFO:PathRAG:Inserting 1 vectors to chunks
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.06batch/s]
INFO:PathRAG:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/1 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks: 100%|██████████| 1/1 [01:09<00:00, 69.94s/chunk]
INFO:PathRAG:Inserting entities into storage...
Inserting entities: 100%|██████████| 25/25 [00:00<00:00, 1419.93entity/s]
INFO:PathRAG:Inserting relationships into storage...
Inserting relationships: 100%|██████████| 12/12 [00:00<00:00, 2034.34relationship/s]
INFO:PathRAG:Inserting 25 vectors to entities
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.11batch/s]
INFO:PathRAG:Inserting 12 vectors to relationships
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.29batch/s]
INFO:PathRAG:Writing graph with 25 nodes, 12 edges


Création de la chaîne Graph RAG en 93 secondes
Création du visuel du graphe de connaissances


In [6]:
from PathRAG import QueryParam
import asyncio

def stream_pathRAG_response(stream_resp):
    async def stream_response():        
        # Process the async generator
        async for chunk in stream_resp:
            print(chunk or "", end="")


    # Run in Streamlit's existing event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(stream_response())

# question="résume ce texte dans sa langue source"
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"

resp=pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]["rag"].query(query= question, param=QueryParam(mode="hybrid", stream=True))

stream_pathRAG_response(resp)

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:PathRAG:kw_prompt result:
INFO:PathRAG:```json
{
  "high_level_keywords": ["Analyse textuelle", "Thèmes principaux", "Questions de discussion"],
  "low_level_keywords": ["Identification des thèmes", "Compréhension de texte", "Questions ouvertes", "Lecture analytique"]
}
```
INFO:PathRAG:Local query uses 25 entites, 15 relations, 1 text units
INFO:PathRAG:Global query uses 17 entites, 12 relations, 1 text units
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


response all ready
## Main Themes and Potential Questions from the Provided Text

The core of the text revolves around the burgeoning field of Artificial Intelligence (AI) and its profound implications for humanity, specifically focusing on creativity, consciousness, and the very definition of intelligence itself.  Several key themes emerge, interwoven through discussions of specific individuals, organizations, and events.

**Central Themes:**

*   **AI and Creativity:** A significant theme is the ability of AI, demonstrated by ChatGPT, to engage in creative endeavors like writing. The literary competition between Hervé Le Tellier and ChatGPT, as highlighted by *Le Nouvel Obs*, serves as a focal point for exploring whether AI can truly *create* or merely *simulate* creativity.  This discussion is underpinned by Le Tellier’s own assessment that ChatGPT may have surpassed his own literary efforts, prompting questions about the future role of human artists.
*   **The Nature of Intelligenc

Renseigner votre clé d'api sur la ligne 20

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.schema.document import Document
from openai import OpenAI, AsyncOpenAI
import asyncio
import json
import re
from dotenv import load_dotenv
import os

load_dotenv("/home/chougar/Documents/GitHub/experiments/.env")

API_KEY_REF=os.getenv("OPENROUTER_API_KEY")


class RAG_hybrid():
    def __init__(self, model):
        self.model=model
        self.retrieved_docs=[]
        self.semantic_retriever_topK=10
        self.sparse_retriever_topK=10
        self.history=[]
        self.llm_client = AsyncOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=API_KEY_REF,
        )
        self.reranker_llm="mistralai/mistral-small-3.1-24b-instruct"
        self.reranker_score_thresh=5
        self.reranked_doc=[]

    def semanticRetriever(self):
        # 1. Semantic Retriever (Chroma + OllamaEmbeddings)
        embeddings = OllamaEmbeddings(model="embeddinggemma")
        chroma_db = Chroma(
            persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
            collection_name=doc_name_hybrid.replace(" ","_"),
            embedding_function=embeddings
        )

        semantic_retriever=chroma_db.as_retriever(search_type="mmr", k=self.semantic_retriever_topK)

        self.chroma_db=chroma_db
        self.semantic_retriever=semantic_retriever
    
    def sparseRetriever(self):
        # 2. Sparse Retriever (TF-IDF)

        # Récupérer TOUS les documents depuis Chroma
        all_data = self.chroma_db.get(include=["documents", "metadatas"])

        # Convertir en liste de `Document` objects pour LangChain
        docs = [
            Document(page_content=text, metadata=meta or {})  # <-- Si meta est None, on met {}
            for text, meta in zip(all_data["documents"], all_data["metadatas"])
        ]

        # Créer le retriever TF-IDF
        sparse_retriever = TFIDFRetriever.from_documents(
            documents=docs,
            k=self.sparse_retriever_topK,
            tfidf_params={"min_df": 1, "ngram_range": (1, 2)}
        )

        self.sparse_retriever= sparse_retriever
    
    def ensembleRetriever(self):
        # 3. Ensemble Retriever (Semantic + Sparse)
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.semantic_retriever, self.sparse_retriever],
            weights=[0.5, 0.5]
        )

        self.ensemble_retriever=ensemble_retriever

    async def reranker(self, results, query):


        async def llm_eval(doc, query):
            system_prompt="""
                You're an expert assistant in reranking documents against a question.
                Your role is to compare the question with a document and give a score from 0 to 10, where:
                0=document out of context, unable to answer the question
                10=highly relevant document, able to answer the question
                                
                The expected final output is the score in json format
                Example:
                ```json{"score": 5}```
                
                Always end your answer with this format                
            """            
            response = await self.llm_client.chat.completions.create(
                model=self.reranker_llm,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"La question est: {query}\n Le document à évaluer est le suivant\n: {doc}" }
                ],
                temperature=0,
            )
            # Post-process to extract only the JSON part if extra text is present
            content = response.choices[0].message.content
            # Try to extract the JSON block if the model adds extra text
            match = re.search(r"\{.*?\}", content, re.DOTALL)
            if match:
                content = match.group(0)

            # extract score
            score=None
            try:
                score=content.replace("```json", "").replace("```", "")
                
                score= json.loads(score)
                score=score["score"]
            except Exception as e:
                print(e)                
            
            return {"content": doc, "score": score}


        tasks=[llm_eval(doc.page_content, query) for doc in results]
        scored_docs= await asyncio.gather(*tasks)
        i=1

        for doc in scored_docs:
          
            print(f'chunk {i} score: {doc["score"]}')
            i+=1

        filtred_docs=[d for d in scored_docs if d["score"]>=self.reranker_score_thresh]
        # print(f"scored docs; \n{scored_docs}")
        self.reranked_doc=filtred_docs

        return filtred_docs

    async def ask_llm(self, query):
        # 5. Final processing step with an LLM (e.g., OpenAI via OpenRouter)

        # init retrievers
        self.semanticRetriever()
        self.sparseRetriever()
        self.ensembleRetriever()

        # retrieve relevant docs
        results = self.ensemble_retriever.get_relevant_documents(query)
        print(f"Nb of retrieved docs: {len(results)}")

        # rerank
        scored_results=await self.reranker(results, query)
        
        # Concatenate retrieved documents for context
        context = "\n".join([f"Fragment: \n{doc['content']}\n" for doc in scored_results])

        print(f"Context lenght: {len(context.split(' '))} words")
        llm_prompt = f"""
            Answer the question based **only** on the provided context.  

            - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
            - If the context lacks relevant information, respond with: "I don't know."  

            ### **Context:**  
            {context}  

            ### **Question:**  
            {query}  

            ### **Answer:**  
            Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
        """

        llm_completion = await self.llm_client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in document Q/A and document synthesis"},
                {"role": "user", "content": llm_prompt}
            ],
            temperature=0.2,
            stream=True
        )

        final_answer = ""
        print("Réponse:\n=========")
        async for chunk in llm_completion:
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                final_answer += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="", flush=True)
        
        self.history+=[
            {"role": "user", 'content': query},
            {"role": "assistant", "content": final_answer}
        ]
        
        return final_answer



In [10]:
rag_hybrid=RAG_hybrid(model="mistralai/mistral-small-3.2-24b-instruct")
# 4. Ask a question
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"
results = await rag_hybrid.ask_llm(question)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Nb of retrieved docs: 5


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


chunk 1 score: 3
chunk 2 score: 7
chunk 3 score: 8
chunk 4 score: 4
chunk 5 score: 7
Context lenght: 444 words


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Réponse:
### **Principaux thèmes du texte :**

1. **Comparaison entre l'intelligence humaine et l'intelligence artificielle (IA)** :
   - Le texte compare les capacités créatives de l'écrivain Hervé Le Tellier et de l'IA ChatGPT, notamment dans la rédaction de nouvelles.
   - Il souligne la capacité de l'IA à produire des œuvres littéraires originales et drôles, remettant en question la supériorité de l'esprit humain dans ce domaine.

2. **Progrès de l'IA et ses implications** :
   - Le texte évoque les avancées rapides de l'IA, notamment depuis l'apparition de ChatGPT en novembre 2022.
   - Il mentionne les nombreuses tâches intellectuelles que l'IA peut désormais accomplir, telles que l'écriture d'articles, la synthèse de documents, le diagnostic de maladies, et la rédaction de scénarios de films.

3. **Philosophie de l'esprit et la nature de l'intelligence** :
   - Le texte aborde la question de savoir si l'intelligence peut être implémentée dans des machines, remettant en question 